# Metaprogramming with Julia

## Scare Quotes

Julia supports *metaprogramming*. This is similar to symbolic programming, where we deal with expressions (like $2+2$) as opposed to values (like $4$).

Normally, Julia takes all code we give it as a set of instructions, and carries them out. If we type `2+2` it will faithfully add those numbers and give us the result.

In [1]:
2+2

4

We can prevent this from happening with quotation marks. Surrounding our code with `"` treats it as a literal string of characters, without seeing it as code at all.

In [2]:
x = "2+2"

"2+2"

We can then explicitly tell Julia to evaluate it later on.

In [3]:
eval(parse(ans))

4

Why go through this complicated way to add numbers? The trick is that when we have the expression `2+2`, we can modify it in various interesting ways. As a simple example, imagine replacing `+` with `-`.

In [4]:
x = replace(x, "+", "-")

"2-2"

In [5]:
eval(parse(x))

0

We don't actually want to work with strings here; Julia has a much more powerful way to quote code, the frowny face operator `:()`.

In [6]:
x = :(2+2)

:(2 + 2)

In [7]:
eval(x)

4

We can quote larger expression, including blocks and entire function definitions. The keyword `quote` is an alternative to `begin` that returns the quoted block.

In larger blocks, Julia will preserve line number information, which appears as a comment.

In [8]:
quote
  x = 2 + 2
  hypot(x, 5)
end

quote  # In[8], line 2:
    x = 2 + 2 # In[8], line 3:
    hypot(x, 5)
end

In [9]:
:(function mysum(xs)
    sum = 0
    for x in xs
      sum += x
    end
  end)

:(function mysum(xs) # In[9], line 2:
        sum = 0 # In[9], line 3:
        for x = xs # In[9], line 4:
            sum += x
        end
    end)

## Fruit of the Forbidden Expression Tree

Strings support "interpolation", which allows us to easily build larger strings from smaller components.

In [10]:
x = "yields falsehood when preceded by its quotation"

"yields falsehood when preceded by its quotation"

In [11]:
"'$x' $x"

"'yields falsehood when preceded by its quotation' yields falsehood when preceded by its quotation"

In [12]:
x = :(2+2)

:(2 + 2)

In [13]:
:($x * $x)

:((2 + 2) * (2 + 2))

In [14]:
eval(ans)

16

## The Root of all Eval

`eval` can do more than just returning a result. What happens if we quote something like a function definition?

In [15]:
ex = :(foo() = println("I'm foo!"))

:(foo() = begin  # In[15], line 1:
            println("I'm foo!")
        end)

It doesn't actually do anything; yet.

In [16]:
foo()

LoadError: [91mUndefVarError: foo not defined[39m

But evaluating `ex` brings `foo` to life!

In [17]:
eval(ex)

foo (generic function with 1 method)

In [18]:
foo()

I'm foo!


Using interpolation, we can construct a function definition on-the-fly; in fact, we can make a whole series of functions at once.

In [19]:
for name in [:foo, :bar, :baz]
  println(:($name() = println($("I'm $(name)!"))))
end

foo() = begin  # In[19], line 2:
        println("I'm foo!")
    end
bar() = begin  # In[19], line 2:
        println("I'm bar!")
    end
baz() = begin  # In[19], line 2:
        println("I'm baz!")
    end


And then bring them to life with `eval`, too.

In [20]:
for name in [:foo, :bar, :baz]
  eval(:($name() = println($("I'm $(name)!"))))
end

In [21]:
bar()

I'm bar!


In [22]:
baz()

I'm baz!


This can be an *extremely* useful trick when wrapping APIs (say, from a C library or over HTTP). APIs often define a list of available functions, so you can grab that and generate the whole wrapper at once! See Clang.jl, TensorFlow.jl, or the Base linear algebra wrappers for examples.

## Original Sin

Here's a more practical example. Consider the following definition of the `sin` function, based on the Taylor series.

$$sin(x) = \sum_{k=1}^{\infty} \frac{(-1)^k}{(1+2k)!} x^{1+2k}$$

In [23]:
mysin(x) = sum((-1)^k/factorial(1.0+2k) * x^(1+2k) for k = 0:5)

mysin (generic function with 1 method)

In [24]:
mysin(0.5), sin(0.5)

(0.4794255386041834, 0.479425538604203)

To see where we are right now, we'll benchmark it.

In [25]:
using BenchmarkTools
@benchmark mysin(0.5)

BenchmarkTools.Trial: 
  memory estimate:  64 bytes
  allocs estimate:  3
  --------------
  minimum time:     677.786 ns (0.00% GC)
  median time:      681.416 ns (0.00% GC)
  mean time:        708.563 ns (0.66% GC)
  maximum time:     13.640 μs (91.01% GC)
  --------------
  samples:          10000
  evals/sample:     154

Right now, this is much slower than it could be. The reason is that we're looping over `k`, which is relatively expensive. It'd be much faster to write out:

In [26]:
mysin(x) = x - x^3/6 + x^5/120 # + ...

mysin (generic function with 1 method)

But this is tedious to write, and no longer looks like the original Taylor series. It's harder to tell if we've made a mistake, and we easily modify it. Is there a way to get the best of both worlds?

How about getting Julia to write out that code for us?

To start with, let's consider a symbolic version of the `+` function.

In [27]:
plus(a, b) = :($a + $b)

plus (generic function with 1 method)

In [28]:
plus(1, 2)

:(1 + 2)

With `plus` we can do more interesting things, like symbolic `sum`:

In [29]:
reduce(+, 1:10)

55

In [30]:
reduce(plus, 1:10)

:(((((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8) + 9) + 10)

In [31]:
eval(ans)

55

Given that, we can also sum over symbolic variables.

In [32]:
reduce(plus, [:(x^2), :x, 1])

:((x ^ 2 + x) + 1)

This gives us an important piece of the puzzle, but we also need to figure out _what_ we're summing. Let's crate a symbolic version of the Taylor series above, which interpolates the value of `k`.

In [33]:
k = 2
:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k)))

:((1 * x ^ 5) / 120)

Now we have one term, we can generate as many as we like.

In [34]:
terms = [:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k))) for k = 0:5]

6-element Array{Expr,1}:
 :((1 * x ^ 1) / 1)         
 :((-1 * x ^ 3) / 6)        
 :((1 * x ^ 5) / 120)       
 :((-1 * x ^ 7) / 5040)     
 :((1 * x ^ 9) / 362880)    
 :((-1 * x ^ 11) / 39916800)

And sum them –

In [35]:
reduce(plus, ans)

:((((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800)

And create a function definition out of it:

In [36]:
:(mysin(x) = $ans)

:(mysin(x) = begin  # In[36], line 1:
            (((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800
        end)

In [37]:
eval(ans)

mysin (generic function with 1 method)

In [38]:
mysin(0.5), sin(0.5)

(0.4794255386041834, 0.479425538604203)

In [39]:
@benchmark mysin(0.5)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     56.507 ns (0.00% GC)
  median time:      56.893 ns (0.00% GC)
  mean time:        58.532 ns (0.00% GC)
  maximum time:     337.294 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     984

On my machine `sin2` takes about 50 *nano*seconds to run – not bad for a naive implementation. If we challenged a photon to a twenty metre sprint, we'd win!

## Beneath the Expression

It's really just a normal tree data structure, and we can take a look inside it.

In [40]:
ex = :(f(x, y))
ex = Expr(:call, :f, :x, :y)
@show ex.head;
@show ex.args;

ex.head = :call
ex.args = Any[:f, :x, :y]


In our example above, we replace `+` with `-` in a string. In an expression, we can do that by altering the `.args` of the expression.

In [41]:
ex = :(2+2)

:(2 + 2)

In [42]:
ex.args[1] = :-
ex

:(2 - 2)

In [43]:
eval(ex)

0

Note that larger, more complex expressions are a bit trickier than this. They are *nested*, which means that the expression `2+3` is contained inside the larger expression `1 + (2 + 3)`.

In [44]:
ex = :(1 + (2 + 3))

:(1 + (2 + 3))

In [45]:
ex.args

3-element Array{Any,1}:
  :+      
 1        
  :(2 + 3)

A package called MacroTools provides a way to deal with this. It shows us *all* sub-expressions in turn, allowing us to decide how we want to change them. It can be thought of as a bit like a find-and-replace operation. Here's an example that finds all integers in an expression, and increments them.

In [46]:
using MacroTools
using MacroTools: postwalk

In [47]:
postwalk(ex) do x
  x isa Integer ? x+1 : x
end

:(2 + (3 + 4))

To get a feel for what's happening, you can use `@show` to see what `postwalk` sees.

In [48]:
postwalk(ex) do x
  @show x
end

x = :+
x = 1
x = :+
x = 2
x = 3
x = :(2 + 3)
x = :(1 + (2 + 3))


:(1 + (2 + 3))

MacroTools also provides tools for *pattern matching* over expressions. `a_ + b_` acts as a template; if the expression provided look like the template, `a` and `b` will match the two things be added. If not, they'll just be `nothing`.

In [49]:
ex = :(2 + 3)
@capture(ex, a_ + b_)

true

In [50]:
a, b

(2, 3)

In [51]:
ex = :(f(2,3))
@capture(ex, a_ + b_)

false

In [52]:
a, b

(nothing, nothing)

We can finally use this to replace _all_ `+`s with `-`s in an expression, rather than just one.

In [53]:
ex = :(3x^2 + (2x + 1))

:(3 * x ^ 2 + (2x + 1))

In [54]:
postwalk(ex) do x
  @capture(x, a_ + b_) ? :($a - $b) : x
end

:(3 * x ^ 2 - (2x - 1))

## Building a Macro

todo adding logging to plus calls, turn into a macro